##### Demo

## Use input size 416:
- file cfg
- yolo3_weight.h5 when convert
- file train.py

In [1]:
import colorsys
import cv2
import dlib
# import face_recognition
from keras import backend as K
from keras.utils import multi_gpu_model
from keras.layers import Input
from keras.models import load_model

import numpy as np
import os
from PIL import Image, ImageFont, ImageDraw
from timeit import default_timer as timer

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image

Using TensorFlow backend.


In [2]:
from imutils.video import WebcamVideoStream

In [3]:
# from threading import Thread
# import cv2
# import imutils
# class WebcamVideoStream:
#     def __init__(self, src=0):
#         self.stream = cv2.VideoCapture(src)
#         self.stream.set(3, 800)
#         self.stream.set(4, 600)
#         (self.grabbed, self.frame) = self.stream.read()
        
#         self.stopped = False
        
#     def start(self):
#         # Start the thread to read frames from the video stream
#         Thread(target=self.update, args=()).start()
#         return self
    
#     def update(self):
#         while True:
#             if self.stopped:
#                 return
            
#             (self.grabbed, self.frame) = self.stream.read()
            
#     def read(self):
#         return self.frame
    
#     def stop(self):
#         self.stopped = True  

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
gpu_num = 1

In [5]:
# model_path = '../logs/human_pose_dataset_1400_416_yolo/trained_weights_final.h5'
model_path = '../logs/human_pose_dataset_2388_size_416_batch_size_4/human_pose_dataset_2388_size_416_batch_size_4.h5'
anchors_path = '../model_data/yolo_anchors.txt'
classes_path = '../model_data/human_pose.txt'
score = 0.4
iou = 0.1
model_image_size = (416, 416)
sess = K.get_session()

In [6]:
# Get class
classes_path = os.path.expanduser(classes_path)
with open(classes_path) as f:
    class_names = f.readlines()

class_names = [c.strip() for c in class_names]

# Anchors
anchors_path = os.path.expanduser(anchors_path)
with open(anchors_path) as f:
    anchors = f.readline()
anchors = [float(x) for x in anchors.split(',')]
anchors = np.array(anchors).reshape(-1, 2)

In [7]:
# Load model
model_path = os.path.expanduser(model_path)
assert model_path.endswith('.h5'), 'Keras model end with file .h5'

num_anchors = len(anchors)
num_classes = len(class_names)

is_tiny_version = num_anchors==6
try:
    yolo_model = load_model(model_path, compile=False)
except:
    if is_tiny_version:
        yolo_model = tiny_yolo_body(Input(shape=(None, None, 3)), num_anchors//2, num_classes)
    else:
        yolo_model = yolo_body(Input(shape=(None, None, 3)), num_anchors//3, num_classes)
    
    yolo_model.load_weights(model_path)
else:
    yolo_model.layers[-1].output_shape[-1] == num_anchors/len(yolo_model.output) * (num_classes + 5), 'Mismatch between model and given anchor and class sizes'
    
print("{} model, anchors, and classes loaded.".format(model_path))

../logs/human_pose_dataset_2388_size_416_batch_size_4/human_pose_dataset_2388_size_416_batch_size_4.h5 model, anchors, and classes loaded.


In [8]:
import numpy as np
import face_recognition

In [9]:
face_encodings_in_room = []
face_names_in_room = []
known_face_encodings_array = np.load("../data/numpy/known_face_encoding.npy")
known_face_names = np.load("../data/numpy/known_face_names.npy")

# Convert nparray -> List to face_encoding
len_of_array_known_face_names = len(known_face_names)
known_face_encodings_array = known_face_encodings_array.reshape(len_of_array_known_face_names, 128)
known_face_encodings = []
for i in range(len_of_array_known_face_names):
    known_face_encodings.append(known_face_encodings_array[i])

In [ ]:
input_image_shape = K.placeholder(shape=(2, ))
boxes, scores, classes = yolo_eval(yolo_model.output, anchors, len(class_names), input_image_shape, score_threshold=score, iou_threshold=iou)
num_frame = 0
font = cv2.FONT_HERSHEY_DUPLEX

# Video capture
video_capture = WebcamVideoStream(src=0).start()

while True:
    num_frame += 1

    # Read video frame and flip camera
    frame = video_capture.read()
    frame = cv2.flip(frame, 1)
    frame_process = np.copy(frame)
    
#     #-------------------------------------------------------#
#     # Face recognition

#     # Convert the image from BGR color to RGB to face_recognition use
#     rgb_frame = frame_process[:, :, ::-1]

#     # Find all the faces and face encodings in the current frame of video
#     face_locations = face_recognition.face_locations(rgb_frame)
#     face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

#     for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
#         distance = face_recognition.face_distance(known_face_encodings, face_encoding)
#         min_distance = np.min(distance)
#         index_point_min = np.argmin(distance)
#         if min_distance < 0.5:
#             name = known_face_names[index_point_min]
#             print(name)
#             cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 3)
#             cv2.putText(frame, name, (left + 6, top - 6), font, 1.0, (255, 255, 0), 1)

#     #-----------------------------------------------------------#
    
    # Detect state standing and sleeping and sitting
    image = Image.fromarray(frame_process)

    # Process detect hand and recognition furniture
    boxed_image = letterbox_image(image, tuple(reversed(model_image_size)))
    image_data = np.array(boxed_image, dtype='float32')
    
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)
    
    out_boxes, out_scores, out_classes = sess.run([boxes, scores, classes],
                                                 feed_dict={
                                                     yolo_model.input: image_data,
                                                     input_image_shape: [image.size[1], image.size[0]],
                                                     K.learning_phase(): 0
                                                 })

    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
        
        label = '{} {:.2f}'.format(predicted_class, score)
        
        top, left, bottom, right = box  
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        
        cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 3)
        cv2.putText(frame, label, (left + 6, top + 30), font, 1.0, (255, 0, 255), 1)

        
#         #-------------------------------------------------------#
#         # Face recognition
#         crop_img = frame_process[top:bottom, left:right]
#         # Convert the image from BGR color to RGB to face_recognition use
#         rgb_frame = crop_img[:, :, ::-1]

#         # Find all the faces and face encodings in the current frame of video
#         face_locations = face_recognition.face_locations(rgb_frame)
#         face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        
#         if not face_encodings:
#             cv2.putText(frame, label, (left + 6, top + 20), font, 1.0, (0, 0, 255), 1)
#         else:
#             for (top1, right1, bottom1, left1), face_encoding in zip(face_locations, face_encodings):
#                 distance = face_recognition.face_distance(known_face_encodings, face_encoding)
#                 min_distance = np.min(distance)
#                 index_point_min = np.argmin(distance)
#                 if min_distance < 0.5:
#                     name = known_face_names[index_point_min]
#                     print(name)
# #                     cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 3)
#                     label = name + ": " + label
#                     cv2.putText(frame, label, (left + 6, top + 20), font, 1.0, (0, 0, 255), 1)
#                 else:
#                     label = "Unknown" + ": " + label
#                     cv2.putText(frame, label, (left + 6, top + 20), font, 1.0, (0, 0, 255), 1)
# #         #-------------------------------------------------------#

    cv2.imshow("Frame", frame)

    #
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

## Test

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread("data/image/hanoi.jpg")

In [ ]:
plt.imshow(img)

In [ ]:
img_crop = img[0:800, 1000:1400]

In [ ]:
plt.imshow(img_crop)

In [ ]:
a = []

In [ ]:
if not a:
    print("true")
else:
    print("False")